In [4]:
import os
import torch

from torchvision.io import read_image
from torchvision.ops.boxes import masks_to_boxes
from torchvision import tv_tensors
from torchvision.transforms.v2 import functional as F

import numpy as np
import pandas as pd
import json

In [19]:
import os
import pathlib

if  "models"  in pathlib.Path.cwd().parts:
   while  "models"  in pathlib.Path.cwd().parts:
    os.chdir('..')
elif  not pathlib.Path('models').exists():
	!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 4050, done.
remote: Counting objects: 100% (4050/4050), done.
remote: Compressing objects: 100% (3078/3078), done.
remote: Total 4050 (delta 1185), reused 2789 (delta 914), pack-reused 0
Receiving objects: 100% (4050/4050), 54.69 MiB | 5.12 MiB/s, done.
Resolving deltas: 100% (1185/1185), done.


In [8]:
train_file = open('data/usdc_train.json')
 
data = json.load(train_file)
 
classes = data['categories']

[{'id': 0, 'name': 'obstacles', 'supercategory': 'none'},
 {'id': 1, 'name': 'biker', 'supercategory': 'obstacles'},
 {'id': 2, 'name': 'car', 'supercategory': 'obstacles'},
 {'id': 3, 'name': 'pedestrian', 'supercategory': 'obstacles'},
 {'id': 4, 'name': 'trafficLight', 'supercategory': 'obstacles'},
 {'id': 5, 'name': 'trafficLight-Green', 'supercategory': 'obstacles'},
 {'id': 6, 'name': 'trafficLight-GreenLeft', 'supercategory': 'obstacles'},
 {'id': 7, 'name': 'trafficLight-Red', 'supercategory': 'obstacles'},
 {'id': 8, 'name': 'trafficLight-RedLeft', 'supercategory': 'obstacles'},
 {'id': 9, 'name': 'trafficLight-Yellow', 'supercategory': 'obstacles'},
 {'id': 10, 'name': 'trafficLight-YellowLeft', 'supercategory': 'obstacles'},
 {'id': 11, 'name': 'truck', 'supercategory': 'obstacles'}]

In [47]:
import torch
from torchvision import models, transforms
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator
from torch.utils.data import DataLoader
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
from torch.optim import SGD
import torchvision.transforms as T
import os

# Load and explore the data
data_path = "data/"
train_annotations_path = os.path.join(data_path, "usdc_train.json")
train_images_path = os.path.join(data_path, "train_images/train_images")

# Load the COCO dataset
dataset = CocoDetection(root=train_images_path, annFile=train_annotations_path)


loading annotations into memory...
Done (t=0.42s)
creating index...
index created!


In [48]:
from albumentations import (
    HorizontalFlip,
    RandomBrightnessContrast,
    Rotate,
    Compose,
    Resize,
    Normalize,
)

def get_train_transform():
    return Compose([
        HorizontalFlip(p=0.5),
        RandomBrightnessContrast(p=0.2),
        Rotate(limit=10, p=0.5),
        Resize(512, 512),  # Adjust size as needed
        Normalize(),  # You can experiment with different normalization techniques
    ], bbox_params={'format': 'coco'})

def get_test_transform():
    return Compose([
        Resize(512, 512),
        Normalize(),
    ], bbox_params={'format': 'coco'})


In [49]:
from torchvision.transforms import functional as F
from torch.utils.data import Dataset

class CustomDataset(Dataset):
    def __init__(self, dataset, transforms=None):
        self.dataset = dataset
        self.transforms = transforms

    def __getitem__(self, idx):
        image, target = self.dataset[idx]
        image = F.to_tensor(image)
        
        if self.transforms is not None:
            augmented = self.transforms(image=image, bboxes=target['boxes'], labels=target['labels'])
            image = augmented['image']
            target['boxes'] = torch.tensor(augmented['bboxes'])
        
        return image, target

    def __len__(self):
        return len(self.dataset)

# Create train and test datasets
train_dataset = CustomDataset(dataset, transforms=get_train_transform())
test_dataset = CustomDataset(dataset, transforms=get_test_transform())

# Create train and test data loaders
train_data_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4)
test_data_loader = DataLoader(test_dataset, batch_size=2, shuffle=False, num_workers=4)
